<a href="https://colab.research.google.com/github/UCR-Research-Computing/Jupyter-Notebooks/blob/main/Stable_Diffusion_Custom_Instance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pwd


/content


In [2]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         187G   52G  136G  28% /
tmpfs            64M     0   64M   0% /dev
shm             5.3G     0  5.3G   0% /dev/shm
tmpfs           6.4G   32K  6.4G   1% /var/colab
/dev/sda1       196G  103G   94G  53% /opt/bin
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware


In [3]:
!nvidia-smi

Fri Oct 21 05:35:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Next, you should install `diffusers==0.4.0` as well `scipy`, `ftfy` and `transformers`.

In [ ]:
!pip install diffusers==0.4.0
!pip install transformers scipy ftfy
!pip install "ipywidgets>=7,<8"

You also need to accept the model license before downloading or using the weights. In this post we'll use model version `v1-4`, so you'll need to  visit [its card](https://huggingface.co/CompVis/stable-diffusion-v1-4), read the license and tick the checkbox if you agree. 

You have to be a registered user in 🤗 Hugging Face Hub, and you'll also need to use an access token for the code to work. For more information on access tokens, please refer to [this section of the documentation](https://huggingface.co/docs/hub/security-tokens).

As google colab has disabled external widgtes, we need to enable it explicitly. Run the following cell to be able to use `notebook_login`

In [5]:
from google.colab import output
output.enable_custom_widget_manager()

Now you can login with your user token. https://huggingface.co/

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [7]:
import torch
from diffusers import StableDiffusionPipeline

# make sure you're logged in with `huggingface-cli login`
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", revision="fp16", torch_dtype=torch.float16)  
#pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4")  

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

Next, let's move the pipeline to GPU to have faster inference.

In [8]:
pipe = pipe.to("cuda")

Using `autocast` will run inference faster because it uses half-precision.

In [24]:
import random

In [ ]:
#prompt = "two cannabis women in a haunted room with a young beautiful blond witch wearing a gown and a sexy magical dark haired beautiful woman witch wearing a cloak smoking cannabis around a ouija board marijuana joints wearing pointy witch hats with spooky lighting summoning ghosts in luis royo art style"
#prompt = "forest goddess underwater, intricate, elegant, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration, magic all around, photorealism, octane render, 8k, cinematic, 35mm"
#prompt = "forest underwater with grass and plants and fish, intricate, elegant, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration, magic all around, photorealism, octane render, 8k, cinematic, 35mm"
#prompt = "underwater forest, realengine"
#prompt = "A digital illustration of a steampunk library with clockwork machines, 4k, detailed, trending in artstation, fantasy vivid colors"
#prompt = "Cluttered house in the woods with mist and magic and stars in the sky, anime, oil painting, high resolution, cottagecore, ghibli inspired, 4k"
#prompt = "A digital illustration of a medieval town, 4k, detailed, trending in artstation, fantasy"
#prompt = "A beautiful medieval town, in the forest along side a rocky fjord, by Josef Thoma, matte painting, trending on artstation HQ"
#prompt = "A beautiful mansion with flowered gardens and a fountain, painting, oil on canvas, 4k, detailed, thomas cole"
#prompt = "wizard, wearing adventure coat, 3d action pose, steampunk, lantern, realengine, trending on artstation HQ"
#prompt = "wizard forest god casting a spell , digital painting , digital art , artstation , devian art , 4k , HD"
#prompt = "elven princess meditating in forest, jade, digital art, trending on artstation, ultra detailed, intricate, sacred geometry, serene, beautiful, photo, realistic, perfect, smooth, 8k"
#prompt = "poseidon humanoid god of the sea, trident, highly detailed, d & d, fantasy, highly detailed, digital painting, trending on artstation, concept art, sharp focus, illustration, art by artgerm and greg rutkowski and magali villeneuve "
#prompt = "rugged forest god the greeman, highly detailed, cinematic, 8 k, by stanley artgermm, tom bagshaw, greg rutkowski, carne griffiths, ayami kojima, beksinski, giger, trending on deviantart, hyper detailed, horror, full of colour "
#prompt = "sexy female poseidon humanoid goddess of the sea, highly detailed, d & d, fantasy, highly detailed, digital painting, trending on artstation, concept art, sharp focus, illustration, art by artgerm and greg rutkowski and magali villeneuve"
#prompt = "sexy female poseidon humanoid goddess of the sea, highly detailed, d & d, fantasy, highly detailed, digital painting, trending on artstation, concept art, sharp focus, illustration, art by artgerm and greg rutkowski and alphonse mucha"
#prompt = "the ocean at night with stars and planets in the sky, studio ghibli, highly detailed, d & d, fantasy, highly detailed, digital painting, trending on artstation, concept art, sharp focus, illustration, art by artgerm and greg rutkowski and magali villeneuve"
#prompt = "watchlights fade from every room, studio ghibli, highly detailed, highly detailed, digital painting, trending on artstation, concept art, sharp focus, illustration"
#prompt = "the ocean at night with stars in the sky, by Josef Thoma, matte painting, trending on artstation HQ"

#theobject = "the ocean at night with stars in the sky"
theobject = "deep space with stars and planets and galaxies"

style_list = ["highly detailed,",
#              "d & d,",
#             "fantasy,",
#             "digital painting,",
             "trending on artstation,",
              "matte painting",
#             "concept art,",
#             "sharp focus,",
#             "illustration,",
#              "magic all around,",
#             "photorealism,",
#             "octane render,",
#             "8k,",
#             "cinematic,",
#             "35mm,",
             ""]
thestyle = " ".join(style_list)


art_list = [\
#           "artgerm,",
#           "luis royo,",
#           "greg rutkowski,",
#           "alphonse mucha,",
#           "studio ghibl,",
            "Josef_Thoma,",
            ""]
theart = " ".join(art_list)


prompt = "%s, %s, %s" % (theobject, thestyle, theart)
cleanprompt = prompt.replace(" ", "_")
print(cleanprompt)
filename = "./images/%s_%s.png" % (cleanprompt, random.randint(0, 9999999)) 

#filename = "./files/StableDiffusion_output_%s.png" % random.randint(0, 9999999) 

with open("prompt_history.txt", "a+") as file_object:
    # Move read cursor to the start of file.
    file_object.seek(0)
    # If file is not empty then append '\n'
    data = file_object.read(100)
    if len(data) > 0 :
        file_object.write("\n")
    # Append text at the end of file
    file_object.write("%s : %s" % (prompt, filename))

#image = pipe(prompt).images[0]
#mage = pipe(prompt, height=768, width=512).images[0]
#image = pipe(prompt, height=512, width=768).images[0]
image = pipe(prompt, height=480, width=856).images[0]
#image = pipe(prompt, height=1920, width=1080).images[0]
image.save(f"%s" % filename)
image

In [ ]:
#
#prompt = "forest underwater with grass and plants and rocks and fish, intricate, elegant, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration, magic all around, photorealism, octane render, 8k, cinematic, 35mm"
#
#prompt = "poseidon humanoid god of the sea, highly detailed, d & d, fantasy, highly detailed, digital painting, trending on artstation, concept art, sharp focus, illustration, art by artgerm and greg rutkowski and magali villeneuve"

theobject = "deep space with stars and planets and galaxies"

style_list = ["highly detailed,",
#              "d & d,",
#             "fantasy,",
#             "digital painting,",
             "trending on artstation,",
              "matte painting",
#             "concept art,",
#             "sharp focus,",
#             "illustration,",
#              "magic all around,",
#             "photorealism,",
#             "octane render,",
#             "8k,",
#             "cinematic,",
#             "35mm,",
             ""]
thestyle = " ".join(style_list)


art_list = [\
#           "artgerm,",
#            "luis royo,",
#          "greg rutkowski,",
#          "alphonse mucha,",
#           "studio ghibl,",
            "Josef_Thoma,",
            ""]
theart = " ".join(art_list)


prompt = "%s, %s, %s" % (theobject, thestyle, theart)
cleanprompt = prompt.replace(" ", "_")
print(cleanprompt)


for x in range(1000):
    
    filename = "./images/%s_%s.png" % (cleanprompt, random.randint(0, 9999999)) 
    
    #image = pipe(prompt).images[0]
    #image = pipe(prompt, height=768, width=512).images[0]
    image = pipe(prompt, height=512, width=856).images[0]
    #image = pipe(prompt, height=1920, width=1080).images[0]
    image.save(f"%s" % filename)
    print(x)

In [ ]:
!tar -cvf images.tar images/*

In [22]:
!rm -rf ./images/*